In [ ]:
""" !-- This file handles part of Task1: Data Scraping and Storage
1. Combine Data: Merge all genre-wise CSVs into a single DataFrame.
2. SQL Storage: Store the merged dataset into an SQL database for querying and future analysis.


**NOTE: MAke Sure to read the comments for steps to be followed """

In [3]:
# Import external modules and libraries for python
import pandas as pd
import glob
%pip install mysql-connector-python
import mysql.connector

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
os.chdir("C:/Users/v-dhramaraj/Desktop/Python/Projects/IMDB _Project1/Data_scrapingAndStorage/IMDB_Project1_CollectedCsvData") # Define Path to store the csv file
# print(os.getcwd())

In [4]:
""" # Combine Data: Merge all genre-wise CSVs into a single DataFrame. """

# Specify the path where your CSV files are located
csv_files = glob.glob("*.csv")

# Read and merge all CSV files into a single DataFrame
movies_all_df = pd.DataFrame()
movies_all_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
# movies_all_df.to_csv(r"C:/Users/v-dhramaraj/Desktop/Python/Projects/IMDB _Project1/Data_scrapingAndStorage/movies_all.csv", index=False)

# Display the merged DataFrame
print(movies_all_df)

                       Title  Rating Votes Duration    Genre
0          Kraven the Hunter     5.5   52K    2h 7m   Action
1               Gladiator II     6.5  225K   2h 28m   Action
2                   Twisters     6.5  165K    2h 2m   Action
3       Sonic the Hedgehog 3     6.9   57K   1h 50m   Action
4      Venom: The Last Dance     6.0  114K   1h 50m   Action
..                       ...     ...   ...      ...      ...
995       The Everything Pot     7.3    32   1h 30m  Romance
996          My Ex's Wedding     8.6    46   1h 35m  Romance
997                     Else     6.1   230   1h 40m  Romance
998  A Merry Royal Christmas     5.2   144   1h 28m  Romance
999   Chainsaws Were Singing     7.2   355   1h 57m  Romance

[1000 rows x 5 columns]


In [ ]:
%pip install mysql-connector-python
import mysql.connector
from sqlalchemy import create_engine

""" NOTE:  !--  Run this code only once, to avoid data duplication/multiple entry --!"""
# Connect to MySQL Workbench and create database if it doesn't exist

# # Create a connection to your MySQL database
# engine = create_engine("mysql+mysqlconnector://dhiviya:******@25@localhost/imdb_movies")
# # Verify connection
# with engine.connect() as connection:
#     print("Connected to MySQL successfully!")
# # Store the DataFrame in MySQL
# movies_all_df.to_sql(name="movies_data", con=engine, if_exists="replace", index=False)


# Connect to MySQL Workbench and create database if it doesn't exist

conn = mysql.connector.connect(
    host="localhost",  # Change this if connecting to a remote server
    user="USERNAME",  # Replace with your MySQL username
    password="PASSWORD",  # Replace with your MySQL password
)

# Create a cursor object
cursor = conn.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS imdb_movies") # Create the database if it doesn't exist
conn.commit() # Commit changes
# Close the initial connection
cursor.close()
conn.close()

# Connect to the new database created //imdb_movies
conn = mysql.connector.connect(
    host="localhost",
    user="USERNAME",
    password="PASSWORD",
    database="imdb_movies"
)
cursor = conn.cursor()
print("Successfully connected to the 'imdb_movies' database!")

# Create a table if it doesn't exist
create_table= """
    CREATE TABLE IF NOT EXISTS imdb_movies (
        id INT AUTO_INCREMENT PRIMARY KEY,
        title VARCHAR(255),
        rating VARCHAR(255),
        votes VARCHAR(255),
        duration VARCHAR(255),
        genre VARCHAR(255)
    )
"""
cursor.execute(create_table)
print("Successfully created table 'imdb_movies'!")

# Insert data into the table

movies_all_df = movies_all_df.infer_objects(copy=False)  # Ensures proper dtype inference
movies_all_df = movies_all_df.fillna(0)  # Replace NaN with 0 for numeric columns
data_to_insert = movies_all_df.values.tolist() # Convert DataFrame to a list of tuples

insert_query = "INSERT INTO imdb_movies(title, rating, votes, duration, genre) VALUES (%s, %s, %s, %s, %s)" # Insert data into the table
cursor.executemany(insert_query, data_to_insert)
conn.commit()

# Always remember to close the connection when done
cursor.close()
conn.close()


^C
Note: you may need to restart the kernel to use updated packages.
Successfully connected to the 'imdb_movies' database!
Successfully created table 'imdb_movies'!


In [ ]:
# Connect to the new database created //imdb_movies
conn = mysql.connector.connect(
    host="localhost",
    user="USERNAME",
    password="PASSWORD",
    database="imdb_movies"
)
cursor = conn.cursor()
print("Successfully connected to the 'imdb_movies' database!")
query = "SELECT * FROM imdb_movies;"
moviesTable_df = pd.read_sql(query, conn)
print(moviesTable_df)
# print(moviesTable_df.head())  # Display first few rows

# cursor.execute("TRUNCATE TABLE imdb_movies;")
# conn.commit()

cursor.close()
conn.close()

Successfully connected to the 'imdb_movies' database!
       id                    title rating votes duration    genre
0       1        Kraven the Hunter    5.5   52K    2h 7m   Action
1       2             Gladiator II    6.5  225K   2h 28m   Action
2       3                 Twisters    6.5  165K    2h 2m   Action
3       4     Sonic the Hedgehog 3    6.9   57K   1h 50m   Action
4       5    Venom: The Last Dance    6.0  114K   1h 50m   Action
..    ...                      ...    ...   ...      ...      ...
995   996       The Everything Pot    7.3    32   1h 30m  Romance
996   997          My Ex's Wedding    8.6    46   1h 35m  Romance
997   998                     Else    6.1   230   1h 40m  Romance
998   999  A Merry Royal Christmas    5.2   144   1h 28m  Romance
999  1000   Chainsaws Were Singing    7.2   355   1h 57m  Romance

[1000 rows x 6 columns]


C:\Users\v-dhramaraj\AppData\Local\Temp\ipykernel_36740\2968407491.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  moviesTable_df = pd.read_sql(query, conn)
